# Pruning Tutorial

In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

## 1.Create a model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

In [3]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[-0.0213, -0.2633,  0.2114],
          [-0.1554, -0.2324, -0.1086],
          [-0.0581,  0.0548, -0.3330]]],


        [[[-0.0444,  0.3029, -0.1271],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0179,  0.2283]]],


        [[[-0.2831,  0.1527,  0.0164],
          [-0.1105, -0.2142,  0.2372],
          [ 0.0442, -0.1857, -0.1708]]],


        [[[-0.2011,  0.1630,  0.2372],
          [ 0.1289,  0.1390, -0.0401],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.1120,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0991,  0.1354]]],


        [[[ 0.0675, -0.0547, -0.1956],
          [ 0.2077,  0.1006, -0.2865],
          [-0.1597, -0.1198, -0.3193]]]], device='cuda:0', requires_grad=True)), ('bias', Parameter containing:
tensor([ 0.0287, -0.3297,  0.1459, -0.0533,  0.2045, -0.3245], device='cuda:0',
       requires_grad=True))]


In [4]:
print(list(module.named_buffers()))

[]


## 2.Pruning a Module

In [6]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [7]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 0.0287, -0.3297,  0.1459, -0.0533,  0.2045, -0.3245], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[-0.0213, -0.2633,  0.2114],
          [-0.1554, -0.2324, -0.1086],
          [-0.0581,  0.0548, -0.3330]]],


        [[[-0.0444,  0.3029, -0.1271],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0179,  0.2283]]],


        [[[-0.2831,  0.1527,  0.0164],
          [-0.1105, -0.2142,  0.2372],
          [ 0.0442, -0.1857, -0.1708]]],


        [[[-0.2011,  0.1630,  0.2372],
          [ 0.1289,  0.1390, -0.0401],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.1120,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0991,  0.1354]]],


        [[[ 0.0675, -0.0547, -0.1956],
          [ 0.2077,  0.1006, -0.2865],
          [-0.1597, -0.1198, -0.3193]]]], device='cuda:0', requires_grad=True))]


In [8]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1.],
          [0., 1., 1.],
          [1., 1., 1.]]],


        [[[0., 1., 0.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 1., 0.],
          [1., 0., 1.],
          [1., 1., 0.]]],


        [[[1., 0., 1.],
          [1., 1., 0.],
          [1., 1., 1.]]],


        [[[0., 1., 1.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 0., 0.],
          [1., 1., 1.],
          [1., 0., 0.]]]], device='cuda:0'))]


In [9]:
print(module.weight)

tensor([[[[-0.0213, -0.0000,  0.2114],
          [-0.0000, -0.2324, -0.1086],
          [-0.0581,  0.0548, -0.3330]]],


        [[[-0.0000,  0.3029, -0.0000],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0000,  0.2283]]],


        [[[-0.2831,  0.1527,  0.0000],
          [-0.1105, -0.0000,  0.2372],
          [ 0.0442, -0.1857, -0.0000]]],


        [[[-0.2011,  0.0000,  0.2372],
          [ 0.1289,  0.1390, -0.0000],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.0000,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0000,  0.1354]]],


        [[[ 0.0675, -0.0000, -0.0000],
          [ 0.2077,  0.1006, -0.2865],
          [-0.1597, -0.0000, -0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


In [10]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x00000184C2325A90>)])


In [11]:
prune.l1_unstructured(module, name="bias", amount=3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [12]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[-0.0213, -0.2633,  0.2114],
          [-0.1554, -0.2324, -0.1086],
          [-0.0581,  0.0548, -0.3330]]],


        [[[-0.0444,  0.3029, -0.1271],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0179,  0.2283]]],


        [[[-0.2831,  0.1527,  0.0164],
          [-0.1105, -0.2142,  0.2372],
          [ 0.0442, -0.1857, -0.1708]]],


        [[[-0.2011,  0.1630,  0.2372],
          [ 0.1289,  0.1390, -0.0401],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.1120,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0991,  0.1354]]],


        [[[ 0.0675, -0.0547, -0.1956],
          [ 0.2077,  0.1006, -0.2865],
          [-0.1597, -0.1198, -0.3193]]]], device='cuda:0', requires_grad=True)), ('bias_orig', Parameter containing:
tensor([ 0.0287, -0.3297,  0.1459, -0.0533,  0.2045, -0.3245], device='cuda:0',
       requires_grad=True))]


In [13]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1.],
          [0., 1., 1.],
          [1., 1., 1.]]],


        [[[0., 1., 0.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 1., 0.],
          [1., 0., 1.],
          [1., 1., 0.]]],


        [[[1., 0., 1.],
          [1., 1., 0.],
          [1., 1., 1.]]],


        [[[0., 1., 1.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 0., 0.],
          [1., 1., 1.],
          [1., 0., 0.]]]], device='cuda:0')), ('bias_mask', tensor([0., 1., 0., 0., 1., 1.], device='cuda:0'))]


In [14]:
print(module.bias)

tensor([ 0.0000, -0.3297,  0.0000, -0.0000,  0.2045, -0.3245], device='cuda:0',
       grad_fn=<MulBackward0>)


In [15]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x00000184C2325A90>), (1, <torch.nn.utils.prune.L1Unstructured object at 0x00000184C232D320>)])


## 3.Iterative Pruning

同一个模块中的同一个参数可以进行多次修剪，各种修剪调用的效果等于一系列应用的各种掩码的组合。新掩码与旧掩码的组合由PruningContainer的compute_mask方法处理。

In [16]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# As we can verify, this will zero out all the connections corresponding to
# 50% (3 out of 6) of the channels, while preserving the action of the
# previous mask.
print(module.weight)

tensor([[[[-0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000,  0.3029, -0.0000],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0000,  0.2283]]],


        [[[-0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[-0.2011,  0.0000,  0.2372],
          [ 0.1289,  0.1390, -0.0000],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.0000,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0000,  0.1354]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


In [17]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # select out the correct hook
        break

print(list(hook))  # pruning history in the container

[<torch.nn.utils.prune.RandomUnstructured object at 0x00000184C2325A90>, <torch.nn.utils.prune.LnStructured object at 0x00000184C2395128>]


In [18]:
print(model.state_dict().keys())

odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


# Remove pruning re-parametrization

In [19]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[-0.0213, -0.2633,  0.2114],
          [-0.1554, -0.2324, -0.1086],
          [-0.0581,  0.0548, -0.3330]]],


        [[[-0.0444,  0.3029, -0.1271],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0179,  0.2283]]],


        [[[-0.2831,  0.1527,  0.0164],
          [-0.1105, -0.2142,  0.2372],
          [ 0.0442, -0.1857, -0.1708]]],


        [[[-0.2011,  0.1630,  0.2372],
          [ 0.1289,  0.1390, -0.0401],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.1120,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0991,  0.1354]]],


        [[[ 0.0675, -0.0547, -0.1956],
          [ 0.2077,  0.1006, -0.2865],
          [-0.1597, -0.1198, -0.3193]]]], device='cuda:0', requires_grad=True)), ('bias_orig', Parameter containing:
tensor([ 0.0287, -0.3297,  0.1459, -0.0533,  0.2045, -0.3245], device='cuda:0',
       requires_grad=True))]


In [20]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 1., 0.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[1., 0., 1.],
          [1., 1., 0.],
          [1., 1., 1.]]],


        [[[0., 1., 1.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], device='cuda:0')), ('bias_mask', tensor([0., 1., 0., 0., 1., 1.], device='cuda:0'))]


In [21]:
print(module.weight)

tensor([[[[-0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000,  0.3029, -0.0000],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0000,  0.2283]]],


        [[[-0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[-0.2011,  0.0000,  0.2372],
          [ 0.1289,  0.1390, -0.0000],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.0000,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0000,  0.1354]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


In [22]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias_orig', Parameter containing:
tensor([ 0.0287, -0.3297,  0.1459, -0.0533,  0.2045, -0.3245], device='cuda:0',
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[-0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000,  0.3029, -0.0000],
          [ 0.3239,  0.2314,  0.3233],
          [ 0.0360, -0.0000,  0.2283]]],


        [[[-0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[-0.2011,  0.0000,  0.2372],
          [ 0.1289,  0.1390, -0.0000],
          [-0.2539,  0.0615,  0.3332]]],


        [[[ 0.0000,  0.2750,  0.2303],
          [-0.1529,  0.1545, -0.1387],
          [-0.2525, -0.0000,  0.1354]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]]]], device='cuda:0', requires_grad=True))]


In [23]:
print(list(module.named_buffers()))

[('bias_mask', tensor([0., 1., 0., 0., 1., 1.], device='cuda:0'))]


# Pruning multiple parameters in a model

In [24]:
new_model = LeNet()
for name, module in new_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])


# Global pruning

In [25]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [26]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 1.85%
Sparsity in conv2.weight: 9.61%
Sparsity in fc1.weight: 22.04%
Sparsity in fc2.weight: 12.15%
Sparsity in fc3.weight: 9.40%
Global sparsity: 20.00%


# Extending torch.nn.utils.prune with custom pruning functions

In [33]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask

In [34]:
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module)
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the
    original (unpruned) parameter is stored in a new parameter named
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module

    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module

In [35]:
model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
